In [ ]:
%logstop
%logstart -rtq ~/.logs/DS_SQL.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Structured Query Language (SQL)
<!-- requirement: data/customers.csv -->
<!-- requirement: data/products.csv -->
<!-- requirement: data/orders.csv -->

SQL is one of the most common computer languages in use for working with data today. It is a standardized language for accessing and manipulating relational databases. While it is relatively limited compared to a general programming language such as Python, it is highly optimized for efficient retrieval and aggregation of data from database tables. Its broad support and use virtually guarantees that any professional data scientist or analyst will encounter SQL eventually. Furthermore, SQL is often the paradigm used to discuss the relational data model, which has implications that apply beyond SQL compliant databases.

We will explore SQL from within Python, which will allow us to work with SQL in a familiar setting and also see opportunities for compatibility between the world of relational databases and data science tools within Python.

## Relational data model

The relational data model for the most part corresponds with our intuitive notion of a table. Each row is a **relation**, usually representing some object, event, or idea. Each column corresponds with an **attribute** which characterizes the relation. In order to reduce redundancy in a database, when creating at able we typically include the minimum amount of attributes required to fully define a relation. This (admittedly vague) guideline is formalized in the idea of [database normalization](https://en.wikipedia.org/wiki/Database_normalization).

For example, considering the following table representing orders from an online retailer.


Customer | ID | Order ID | Product ID | Price | Delivery Address | Billing Address
:-------:|:--:|:--------:|:----------:|:-----:|:----------------:|:---------------:
   Omar  | 435|   62353  |    103     |  6.95 |  ***** Munich, Germany | ***** Berlin, Germany |
   Omar  | 435|   62353  |    4028    |  35.50|  ***** Tunis, Tunisia  | ***** Berlin, Germany |
  Stuart |5692|   64598  |    103     |  6.95 |  ***** Dover, UK | ***** Dover, UK |
  Vidhya |6127|   64921  |    3158    | 101.99|  ***** Mumbai, India | ***** Mumbai, India |
  Vidhya |6127|   64989  |    2561    | 21.35 |  ***** Mumbai, India | ***** Mumbai, India |
  Vidhya |6127|   64989  |     89     | 16.95 |  ***** Mumbai, India | ***** Mumbai, India |
  Stuart |5692|   65271  |    103     |  6.95 |  ***** Dover, UK | ***** Dover, UK |  

In the above table we've reproduced many values several times such as customer names and IDs, addresses, prices, etc. We could break up this table into several smaller tables in which relations contain the minimal amount of attributes needed to define the relation. For instance, we may have a table for customers, a table for products, and a table for orders.

  Customer | ID | Billing Address
 :--------:|:--:|:---------------:
    Omar   | 435| ***** Berlin, Germany
   Stuart  |5692| ***** Dover, UK
   Vidhya  |6127| ***** Mumbai, India

 Product ID | Price
:----------:|:-----:
    103     |  6.95
   4028     | 35.50
   3158     | 101.99
   2561     | 21.35
    89      | 16.95

 Order ID | Customer ID | Product ID | Delivery Address
:--------:|:-----------:|:----------:|:----------------:
   62353  |     435     |    103     | ***** Munich, Germany
   62353  |     435     |    4028    | ***** Tunis, Tunisia
   64598  |    5692     |    103     | ***** Dover, UK
   64921  |    6127     |    3158    | ***** Mumbai, India
   64989  |    6127     |    2561    | ***** Mumbai, India
   64989  |    6127     |     89     | ***** Mumbai, India
   65271  |    5692     |    103     | ***** Dover, UK

Before we were storing 7 rows x 7 columns = 49 cells; now we're storing only 7 x 4 + 5 x 2 + 3 x 3 = 47 cells. This may not seem like a huge improvement, but realistically an online retailer may have millions of orders of a particular product. Reproducing the price in every order rather than storing it once per product could be quite costly when scaled up.

Let's explore how this would be implemented in SQL. We'll use `sqlite`, a basic SQL database manager that is useful for small data analysis and instructional purposes.   

## Loading data in SQL

In [ ]:
%load_ext sql
%sql sqlite:///testdb.sqlite

In [ ]:
%%sql
--# The %%sql magic tells Jupyter to interpret this cell as SQL
--# In SQL comments begin with "--" (we add # to take advantage of Jupyter's syntax highlighting)

--# Since we're starting a new example, let's delete any existing tables
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS orders;

--# Now let's make our tables
CREATE TABLE customers (
    id                 INTEGER PRIMARY KEY NOT NULL,
    name               TEXT NOT NULL,
    billing_address    TEXT NOT NULL
);

CREATE TABLE products (
    id                 INTEGER PRIMARY KEY NOT NULL,
    price              NUMBER NOT NULL
);

CREATE TABLE orders (
    id                 INTEGER NOT NULL,
    customer_id        NUMBER NOT NULL,
    product_id         NUMBER NOT NULL,
    delivery_address   TEXT NOT NULL,
    FOREIGN KEY(customer_id) REFERENCES customers(id),
    FOREIGN KEY(product_id) REFERENCES products(id)
);

Our tables are initially empty, but we have defined the **schema** or structure of the tables. We've specified certain options in our schema, such as the fact that we do not accept null values in any field and that certain fields are unique primary keys. Many more options are possible, including setting default values for fields that could otherwise be null or instructing SQL to automatically assign incrementing values. If you haven't gotten the sense already, database architecture is an extensive subject!

We can inspect the table by using `SELECT`.

In [ ]:
%%sql

SELECT * FROM customers;

Let's `INSERT` data into our tables.

We have to be careful to do this in a certain order; when we defined the `orders` table, we defined a relationship between the `customer_id` and `product_id` attributes and the `id` attributes in the `customer` and `product` tables respectively. We can only `INSERT`  data into the orders table once the appropriate customers and products exist in their tables.

In [ ]:
%%sql

--# Starting with customers

INSERT INTO customers (id, name, billing_address)
    VALUES (435, 'Omar', 'Berlin, Germany'), (5692, 'Stuart', 'Dover, UK'), (6127, 'Vidhya', 'Mumbai, India');

INSERT INTO products (id, price)
    VALUES (103, 6.95), (4028, 35.5), (3158, 101.99), (2561, 21.35), (89, 16.95);

INSERT INTO orders (id, customer_id, product_id, delivery_address)
    VALUES (62353, 435, 103, 'Munich, Germany'), (62353, 435, 4028, 'Tunis, Tunisia');

INSERT INTO orders (id, customer_id, product_id, delivery_address)
    VALUES (64598, 5692, 103, 'Dover, UK'), (65271, 5692, 103, 'Dover, UK');

INSERT INTO orders (id, customer_id, product_id, delivery_address)
    VALUES (64921, 6127, 3158, 'Mumbai, India'), (64989, 6127, 2561, 'Mumbai, India'), (64989, 6127, 89, 'Mumbai, India');

Let's confirm that our tables have been updated with the data from our example.

In [ ]:
%%sql

SELECT * FROM customers;

In [ ]:
%%sql

SELECT * FROM products;

In [ ]:
%%sql

SELECT * FROM orders;

Databases are commonly used for persistent data storage, and therefore it is common to add or remove rows as new data is created (e.g. someone places an order) or destroyed (e.g. a product is discontinued). This may be performed automatically via an application's **database connection**; we will use database connections later in this notebook. However, in the mean time we will load in a larger version of the above data set from file for analysis.

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("testdb.sqlite")
customers = pd.read_csv('data/customers.csv')
products = pd.read_csv('data/products.csv')
orders = pd.read_csv('data/orders.csv')

customers.to_sql("customers", conn, index=False, if_exists="replace")
products.to_sql("products", conn, index=False, if_exists="replace")
orders.to_sql("orders", conn, index=False, if_exists="replace")

## Filtering and sorting data

Filtering is principally accomplished using the `WHERE` command.

In [ ]:
%%sql

SELECT id, delivery_country FROM orders
WHERE delivery_country = 'India'
LIMIT 10;

In [ ]:
%%sql

SELECT * FROM products
WHERE price > 20
LIMIT 10;

We can additionally combine `WHERE` with `LIKE` for pattern matching an `IN` for membership.

In [ ]:
%%sql

SELECT id, delivery_country FROM orders
WHERE delivery_country like 'S%'
LIMIT 10;

In [ ]:
%%sql

SELECT * FROM orders
WHERE customer_id IN (10, 200, 400);

We can also combine them with the usual logical operators: `AND`, `OR`, and `NOT`.

In [ ]:
%%sql

SELECT * FROM orders
WHERE customer_id IN (10, 200, 400)
AND delivery_country NOT IN ('Madagascar', 'Canada');

In [ ]:
%%sql

SELECT * FROM products
WHERE price < 10 OR price > 30
LIMIT 10;

To sort our results, we can `ORDER BY` one or more columns. We can also choose whether we sort in ascending (`ASC`) or descending (`DESC`) order. SQL sorts in ascending order by default.

In [ ]:
%%sql

SELECT * FROM orders
ORDER BY customer_id
LIMIT 10;

In [ ]:
%%sql

SELECT * FROM orders
ORDER BY customer_id ASC, product_id DESC
LIMIT 10;

## Data aggregation

Most of the standard data aggregation functions are available in SQL (`COUNT`, `SUM`, `DISTINCT`, `MAX`, etc.) although exactly what is available and what it is called varies by dialect.

In [ ]:
%%sql

SELECT AVG(price), MAX(price) FROM products;

As usual, we are often interested in aggregating our data within certain groups. As in Pandas, we will use `GROUP BY` to accomplish this. Remember -- if we are performing a `groupby`, any other attributes we select must be aggregated by some aggregation function.

In [ ]:
%%sql

SELECT delivery_country, COUNT(DISTINCT(id)) FROM orders
GROUP BY delivery_country
LIMIT 10;

## Joining tables together

Since we have split up our data among several tables to reduce redundancy, we will have to join tables together to compute certain values we might be interested in. For instance, how might we calculate the total revenue from all orders? We could take a sum of the price associated with each item in each order, but in order to do so, we must `JOIN` the `products` table to the `orders` table `ON` the shared attribute: `product_id` (from the `orders` table) and `id` (from the `products` table).

Since joins involve fields from multiple tables, we'll frequently alias a table `AS` some abbreviation to save ourselves some typing.

In [ ]:
%%sql

SELECT SUM(p.price) FROM orders AS o
JOIN products AS p ON o.product_id = p.id;

There are often several ways to perform a join. We can usually rely on our database management software to work out the details of the most efficient way to perform the join, although there are exceptions.

In [ ]:
%%sql

SELECT SUM(p.price)
FROM orders o, products p
WHERE p.id = o.product_id;

In [ ]:
%%sql

SELECT c.name, SUM(p.price) total
FROM orders o, products p, customers c
WHERE p.id = o.product_id AND c.id = o.customer_id
GROUP BY c.id
ORDER BY total
LIMIT 10;

Let's try something more complex. Let's find the total amount of money spent on orders that are shipped internationally for each `billing_country`.

In [ ]:
%%sql

SELECT shp.bill, SUM(shp.rev) spent
FROM (SELECT c.billing_country bill, o.delivery_country deliver, SUM(p.price) rev
      FROM orders o, customers c, products p
      WHERE o.customer_id = c.id AND o.product_id = p.id
      GROUP BY bill, deliver
      HAVING bill != deliver) shp
GROUP BY shp.bill
ORDER BY spent DESC
LIMIT 10;

The above example makes use of a subquery. Subqueries are often used for constructing intermediate tables that we may use in the computation of a larger query, and are frequently used as part of joins or to perform joins.

## Connecting to a database from Python

In order to load our example data into our database, we created a **database connection**. We then read in our data files with Pandas, and pushed them through the connection to the database. We could have read this data directly into SQL, but database connections allow us to pass data between Python and SQL, allowing web applications or machine learning models operating in Python easy access to persistent databases.

In our case, we used the `sqlite3` module because we are creating a connection to SQLite. There are other connectors for other dialects such a `psycopg2` for PostgreSQL and `mysql` for MySQL. Other packages such as `SQLAlchemy` provide connectors as well as object-relation mapping (ORM), which we will discuss later.

Database connections will typically resemble the example set above

```python
conn = sqlite3.connect("testdb.sqlite")
```

possibly using a URL for connecting to a remotely hosted database and extra parameters for authentication. We can combine the connection with Pandas methods for [reading from](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html) and [writing to](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) SQL.

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*